https://pyscipopt.readthedocs.io/_/downloads/en/latest/pdf/

In [1]:
# from pyscipopt import Model
# # Create the SCIP model
# model = Model("MIQCP Example")

# # Define variables
# x = model.addVar("x", vtype="BINARY")
# y = model.addVar("y", vtype="INTEGER", lb=0, ub=10)
# z = model.addVar("z", vtype="CONTINUOUS", lb=-5, ub=5)

# # Set objective function
# model.setObjective(2*x + 3*y - z, sense="maximize")

# # Add constraints
# model.addCons(x + y + z <= 15, "c1")
# model.addCons(x + 2*y + z*z <= 20, "c2") # Quadratic constraint
# model.addCons(x - y >= 1, "c3")

# # Solve the model
# model.optimize()

# print(model.getStatus())

# # Print solution
# if model.getStatus() == "optimal":
#     print("Optimal solution found:")
#     print("x =", model.getVal(x))
#     print("y =", model.getVal(y))
#     print("z =", model.getVal(z))
#     print("Objective value =", model.getObjVal())
# else:
    # print("No solution found or optimization failed.")

Using example from https://pyscipopt.readthedocs.io/en/latest/tutorials/model.html to check that it runs / my environment is set up correctly:

In [2]:
# from pyscipopt import Model
# scip = Model()
# x = scip.addVar(vtype='C', lb=0, ub=None, name='x')
# y = scip.addVar(vtype='C', lb=0, ub=None, name='y')
# z = scip.addVar(vtype='C', lb=0, ub=80, name='z')
# cons_1 = scip.addCons(x + y <= 5, name="cons_1")
# cons_1 = scip.addCons(y + z >= 3, name="cons_2")
# cons_1 = scip.addCons(x + y == 5, name="cons_3")
# scip.setObjective(2 * x + 3 * y - 5 * z, sense="minimize")
# scip.optimize()

# solve_time = scip.getSolvingTime()
# num_nodes = scip.getNTotalNodes() # Note that getNNodes() is only the number of nodes for the current run (resets at restart)
# obj_val = scip.getObjVal()
# for scip_var in [x, y, z]:
#     print(f"Variable {scip_var.name} has value {scip.getVal(scip_var)}")

Now adapting it for my problem:

In [3]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
scip = Model()

User Inputs:

In [5]:
#Dimensions of BOM items:
a = np.array([5,5,10,9,10]) #longest dimension
b = np.array([4,5,1,5,5]) #mid dimension
c = np.array([0.5,0.5,0.5,0.5,0.5]) #smallest dimension

#Dimensions of UNIQUE stock items available for purchase:
l = np.array([15,15]) #longest dimension
w = np.array([10,15]) #mid dimension
h = np.array([0.5,0.5]) #smallest dimension

#Prices of UNIQUE stock items available for purchase:
prices = [30,40]

#Variable dimension info:
n = len(a) #Total number of items on BOM
m = n * len(l) #Upper limit of stock items we'll have to buy

#Adjust l, w, h to be proper dimensions (using n):
new_l = []
for length in l:
    for i in range(n):
        new_l.append(length)
l = new_l

new_w = []
for width in w:
    for i in range(n):
        new_w.append(width)
w = new_w

new_h = []
for height in h:
    for i in range(n):
        new_h.append(height)
h = new_h

#Adjust dimensions of p / replicate each stock board n-1 times:
p = []
for price in prices:
    for i in range(n):
        p.append(price)

p = np.array(p)

#https://pyscipopt.readthedocs.io/en/latest/tutorials/vartypes.html#variable-types

Decision Variables:

In [6]:
u_array = np.zeros((n, m), dtype=object)
for i in range(n):
    for j in range(m):
        u_array[i][j] = scip.addVar(vtype='B', name=f"u_{i}_{j}")

x_array = np.zeros((n,),dtype=object)
for i in range(n):
    x_array[i] = scip.addVar(vtype='C',lb=0, ub=None, name=f"x_{i}")

y_array = np.zeros((n,),dtype=object)
for i in range(n):
    y_array[i] = scip.addVar(vtype='C',lb=0, ub=None, name=f"y_{i}")

r_array = np.zeros((n,),dtype=object)
for i in range(n):
    r_array[i] = scip.addVar(vtype='B', name=f"r_{i}")

Intermediate Variables:

In [7]:
q_array = np.zeros((m,),dtype=object)
for j in range(m):
    q_array[j] = scip.addVar(vtype='B', name=f"q_{j}")

s_array = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        s_array[i][k] = scip.addVar(vtype='B', name=f"s_{i}_{k}")

t_array = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        t_array[i][k] = scip.addVar(vtype='B', name=f"t_{i}_{k}")

v_array = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        v_array[i][k] = scip.addVar(vtype='B', name=f"v_{i}_{k}")

d_array = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        d_array[i][k] = scip.addVar(vtype='B', name=f"d_{i}_{k}")

f_array = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        f_array[i][k] = scip.addVar(vtype='B', name=f"f_{i}_{k}")

g_array = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        g_array[i][k] = scip.addVar(vtype='B', name=f"g_{i}_{k}")

Constraints:

In [8]:
#1. All BOM items must be cut exactly once / from exactly one stock board:
constr_1 = np.zeros((n,), dtype=object)
for i in range(n):
    constr_1[i] = scip.addCons(quicksum(u_array[i][j] for j in range(m)) == 1, name=f"constr_1_{i}")

In [9]:
#2. The thickness (smallest dimension) of each BOM item must match that of the stock item from which it's cut
#2a.
constr_2a = np.zeros((n,m), dtype=object)
for i in range(n):
    for j in range(m):
        constr_2a[i][j] = scip.addCons(u_array[i][j] <= c[i]/h[j], name=f"constr_2a_{i}_{j}")

#2b.
constr_2b = np.zeros((n,m), dtype=object)
for i in range(n):
    for j in range(m):
        constr_2b[i][j] = scip.addCons(u_array[i][j] <= h[j]/c[i], name=f"constr_2b_{i}_{j}")

In [10]:
#3. If any BOM items are planned to be cut from stock board j, we must buy stock board j:
constr_3 = np.zeros((m,), dtype=object)
for j in range(m):
    constr_3[j] = scip.addCons(q_array[j] >= quicksum(u_array[i][j] for i in range(n))/n, name="constr_3_{j}")

In [ ]:
#4. BOM items cannot exceed the boundaries of the stock board from which they're cut:
constr_4a = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4a[i][k] = scip.addCons(v_array[i][k] == s_array[i][k] + t_array[i][k], name="constr_4a_{i}_{k}")

constr_4c = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4c[i][k] = scip.addCons(s_array[i][k] <= (y_array[k] + 1) / (y_array[i] + (1-r_array[i])*a[i] + r_array[i]*b[i]+1), name="constr_4c_{i}_{k}")

constr_4d = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4d[i][k] = scip.addCons(s_array[i][k] >= y_array[k] - (y_array[i] + (1-r_array[i])*a[i] + r_array*b[i]), name="constr_4d_{i}_{k}")


Objective Function:

In [12]:
scip.setObjective(quicksum(p[j]*q_array[j] for j in range(m)), sense="minimize")

In [13]:
scip.optimize()

In [14]:
solve_time = scip.getSolvingTime()
num_nodes = scip.getNTotalNodes() # Note that getNNodes() is only the number of nodes for the current run (resets at restart)
obj_val = scip.getObjVal()

print("solve_time: ",solve_time)
print("num_nodes: ",num_nodes)
print("obj_val: ",obj_val)

# for scip_var in [x, y, z]:
#     print(f"Variable {scip_var.name} has value {scip.getVal(scip_var)}")

solve_time:  0.0
num_nodes:  1
obj_val:  30.0
